# Installation

In [1]:
!pip install yfinance

In [2]:
!apt-get update && sudo apt-get install cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [633 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,512 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,280 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,035 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InReleas

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
from finrl.main import check_and_make_directories
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv

check_and_make_directories([TRAINED_MODEL_DIR])

import itertools
# Add imports for meta-learning and reinforcement learning libraries
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
#import torchvision.transforms as transforms

import glob, random
from collections import OrderedDict

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"DEVICE = {device}")
random_seed = 0
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/dtypes.py:35: DeprecationWarning: ml_dtypes.float8_e4m3b11 is deprecated. Use ml_dtypes.float8_e4m3b11fnuz
  from tensorflow.tsl.python.lib.core import pywrap_ml_dtypes


DEVICE = cuda


In [4]:
!pip install stable-baselines[mpi]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.3 MB/s eta 0:00:00
  Created wheel for mpi4py: filename=mpi4py-3.1.5-cp310-cp310-linux_x86_64.whl size=2746490 sha256=b44a177b381e8df260852452ef29f999810e827139287eab9b6d4e9e9bed417e
  Stored in directory: /root/.cache/pip/wheels/18/2b/7f/c852523089e9182b45fca50ff56f49a51eeb6284fd25a66713
Successfully built mpi4py
  Attempting uninstall: ale-py
    Found existing installation: ale-py 0.8.1
    Uninstalling ale-py-0.8.1:
      Successfully uninstalled ale-py-0.8.1


In [5]:
!pip install tensorflow

In [6]:
import yfinance as yf
from stockstats import StockDataFrame as Sdf

import pandas as pd
import matplotlib.pyplot as plt

import gym
from stable_baselines3 import PPO, DDPG, A2C, TD3
from stable_baselines3.common.vec_env import DummyVecEnv

# Data Acquisition and Preprocessing

In [7]:
config_tickers.DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [8]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2021-10-29'

In [9]:
df_raw = YahooDownloader(start_date=TRAIN_START_DATE, end_date=TRADE_END_DATE, ticker_list=config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

# Feature Engineering

In [10]:
fe = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list=INDICATORS,
    use_vix=True,
    use_turbulence=True,
    user_defined_feature=False
)
df_processed = fe.preprocess_data(df_raw)
df_processed.head()

Successfully added technical indicators
[*********************100%%**********************]  1 of 1 completed
Shape of DataFrame:  (3228, 8)
Successfully added vix
Successfully added turbulence index


,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,3.067143,3.251429,3.041429,2.747391,746015200,AAPL,4,0.0,2.969345,2.641387,100.0,66.666667,100.0,2.747391,2.747391,39.189999,0.0
1,2009-01-02,58.590000,59.080002,57.750000,42.737888,6547900,AMGN,4,0.0,2.969345,2.641387,100.0,66.666667,100.0,42.737888,42.737888,39.189999,0.0
2,2009-01-02,18.570000,19.520000,18.400000,15.193845,10955700,AXP,4,0.0,2.969345,2.641387,100.0,66.666667,100.0,15.193845,15.193845,39.189999,0.0
3,2009-01-02,42.799999,45.560001,42.779999,33.941097,7010200,BA,4,0.0,2.969345,2.641387,100.0,66.666667,100.0,33.941097,33.941097,39.189999,0.0
4,2009-01-02,44.910000,46.980000,44.709999,31.093407,7117200,CAT,4,0.0,2.969345,2.641387,100.0,66.666667,100.0,31.093407,31.093407,39.189999,0.0


In [11]:
list_ticker = df_processed["tic"].unique().tolist()
list_date = list(pd.date_range(df_processed['date'].min(), df_processed['date'].max()).astype(str))
combination = list(itertools.product(list_date, list_ticker))

processed_full = pd.DataFrame(combination, columns=["date","tic"]).merge(df_processed, on=["date","tic"], how="left")
processed_full = processed_full[processed_full['date'].isin(df_processed['date'])]
processed_full = processed_full.sort_values(['date', 'tic'])
processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.747391,746015200.0,4.0,0.000000,2.969345,2.641387,100.000000,66.666667,100.000000,2.747391,2.747391,39.189999,0.000000
1,2009-01-02,AMGN,58.590000,59.080002,57.750000,42.737888,6547900.0,4.0,0.000000,2.969345,2.641387,100.000000,66.666667,100.000000,42.737888,42.737888,39.189999,0.000000
2,2009-01-02,AXP,18.570000,19.520000,18.400000,15.193845,10955700.0,4.0,0.000000,2.969345,2.641387,100.000000,66.666667,100.000000,15.193845,15.193845,39.189999,0.000000
3,2009-01-02,BA,42.799999,45.560001,42.779999,33.941097,7010200.0,4.0,0.000000,2.969345,2.641387,100.000000,66.666667,100.000000,33.941097,33.941097,39.189999,0.000000
4,2009-01-02,CAT,44.910000,46.980000,44.709999,31.093407,7117200.0,4.0,0.000000,2.969345,2.641387,100.000000,66.666667,100.000000,31.093407,31.093407,39.189999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135773,2021-10-27,UNH,454.640015,460.440002,453.480011,439.670868,3520400.0,2.0,11.306588,450.024449,360.941887,65.085936,174.365131,51.571330,402.508753,402.792323,16.980000,145.284945
135774,2021-10-27,V,224.750000,224.750000,215.660004,212.099335,22958100.0,2.0,0.013797,232.994211,214.822635,41.737211,-153.682031,30.337021,223.018859,225.227716,16.980000,145.284945
135775,2021-10-27,VZ,53.169998,53.200001,52.470001,46.484341,15007400.0,2.0,-0.215911,48.194485,45.191918,44.604886,-50.004225,8.504850,46.923545,47.527329,16.980000,145.284945
135776,2021-10-27,WBA,48.450001,48.459999,47.090000,41.611919,5652000.0,2.0,-0.014508,44.215244,40.487787,46.167782,-62.027001,5.045608,42.551244,42.866391,16.980000,145.284945


In [12]:
df_train = data_split(processed_full, TRAIN_START_DATE, TRAIN_END_DATE)
df_test = data_split(processed_full, TRADE_START_DATE, TRADE_END_DATE)
print(len(df_train))
print(len(df_test))

83897
9715


In [13]:
print(df_test[:3])

         date   tic        open        high         low       close  \
0  2020-07-01  AAPL   91.279999   91.839996   90.977501   89.132996   
0  2020-07-01  AMGN  235.520004  256.230011  232.580002  228.754761   
0  2020-07-01   AXP   95.250000   96.959999   93.639999   90.062149   

        volume  day      macd     boll_ub     boll_lb     rsi_30      cci_30  \
0  110737200.0  2.0  2.988730   91.903737   79.491806  62.807122  107.505365   
0    6575800.0  2.0  3.463789  221.366394  190.709574  61.279645  270.271588   
0    3301000.0  2.0 -0.380609  107.996624   85.830222  48.504815  -66.283273   

       dx_30  close_30_sma  close_60_sma        vix  turbulence  
0  29.730532     83.213408     77.050535  28.620001   53.068342  
0  46.806139    204.660121    205.682194  28.620001   53.068342  
0   3.142448     95.107021     88.701872  28.620001   53.068342  


### Separate different stocks

In [14]:
# Extracting the unique tickers from the dataframe
unique_tickers = df_train['tic'].unique()

# Creating an array of dataframes, each containing data for a specific ticker
dataframes_array_train = [df_train[df_train['tic'] == ticker] for ticker in unique_tickers]

# Displaying the tickers and the corresponding index in the array
ticker_index_mapping = {ticker: index for index, ticker in enumerate(unique_tickers)}
ticker_index_mapping


{'AAPL': 0,
 'AMGN': 1,
 'AXP': 2,
 'BA': 3,
 'CAT': 4,
 'CRM': 5,
 'CSCO': 6,
 'CVX': 7,
 'DIS': 8,
 'GS': 9,
 'HD': 10,
 'HON': 11,
 'IBM': 12,
 'INTC': 13,
 'JNJ': 14,
 'JPM': 15,
 'KO': 16,
 'MCD': 17,
 'MMM': 18,
 'MRK': 19,
 'MSFT': 20,
 'NKE': 21,
 'PG': 22,
 'TRV': 23,
 'UNH': 24,
 'V': 25,
 'VZ': 26,
 'WBA': 27,
 'WMT': 28}

# Catagorizing

In [15]:
dataframes_array_train[1]

## dataframes_array_train[i] => the ith stock data (before process)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AMGN,58.590000,59.080002,57.750000,42.737888,6547900.0,4.0,0.000000,2.969345,2.641387,100.000000,66.666667,100.000000,42.737888,42.737888,39.189999,0.000000
1,2009-01-05,AMGN,59.220001,59.650002,58.000000,43.216061,10940100.0,0.0,0.010728,43.653212,42.300737,100.000000,66.666667,100.000000,42.976974,42.976974,39.080002,0.000000
2,2009-01-06,AMGN,60.040001,60.150002,57.750000,42.266964,8974000.0,1.0,-0.015762,43.689410,41.791198,32.751607,-5.007907,100.000000,42.740304,42.740304,38.560001,0.000000
3,2009-01-07,AMGN,57.830002,58.750000,57.740002,42.165554,5725400.0,2.0,-0.031085,43.561434,41.631799,30.485558,-109.963001,97.952347,42.596617,42.596617,43.389999,0.000000
4,2009-01-08,AMGN,58.060001,59.279999,58.020000,42.933506,5576700.0,3.0,-0.008198,43.552223,41.775766,54.919819,35.559063,98.704137,42.663995,42.663995,42.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,2020-06-24,AMGN,235.210007,237.000000,231.399994,208.481400,1972900.0,2.0,1.530311,213.294581,192.950432,53.126174,82.816906,17.727399,204.120564,203.127013,33.840000,10.120450
2889,2020-06-25,AMGN,231.500000,235.440002,230.389999,210.812683,2080300.0,3.0,1.794469,214.288256,192.941287,54.191875,87.275025,14.780337,204.116747,203.631277,32.220001,22.286126
2890,2020-06-26,AMGN,235.369995,236.860001,231.820007,208.777283,3613300.0,4.0,1.818614,214.631715,192.879380,53.101466,94.686478,17.870239,204.010942,204.174650,34.730000,68.833763
2891,2020-06-29,AMGN,233.399994,234.139999,229.979996,207.629532,1938000.0,0.0,1.725247,214.975339,192.952697,52.485418,72.254378,12.233549,203.752705,204.534572,31.780001,36.823301


In [16]:
dataframes_array_train[0].to_csv('ticker_0_train.csv', index=False)

In [17]:
type(dataframes_array_train[0]['date'])

pandas.core.series.Series

In [18]:
dataframes_array_train[0]['date']

0       2009-01-02
1       2009-01-05
2       2009-01-06
3       2009-01-07
4       2009-01-08
           ...    
2888    2020-06-24
2889    2020-06-25
2890    2020-06-26
2891    2020-06-29
2892    2020-06-30
Name: date, Length: 2893, dtype: object

In [19]:
from sklearn.linear_model import LinearRegression
slope_threshold_up = 0.01
slope_threshold_down = -slope_threshold_up
upward_segments = []
downward_segments = []
bumpy_segments = []

In [29]:
def load_data(data):
    # Load the stock price data from a CSV file
    # Convert 'date' column to datetime format
    data['date'] = pd.to_datetime(data['date'])
    return data
def calculate_slope(x, y):
    # Fit a linear regression model and return the slope
    model = LinearRegression().fit(x.reshape(-1, 1), y)
    return model.coef_[0]

def categorize_trend(segment):
    # Determine the type of trend in the segment
    # x values are the number of days since the start of the segment
    x_values = np.array((segment['date'] - segment['date'].iloc[0]).dt.days)
    slope = calculate_slope(x_values, segment['close'].values)
    if slope > slope_threshold_up: # slope_threshold is a predefined threshold for slope
        return "upward"
    elif slope < slope_threshold_down:
        return "downward"
    else:
        return "bumpy"

def analyze_trends(data):
    # Break the data into 3-month segments and analyze trends
    trend_categories = []
    # Ensure the first date is a datetime object
    start_date = pd.to_datetime(data['date'].iloc[0])
    while start_date < pd.to_datetime(data['date'].iloc[-1]):
        # Determine the end date for the 3-month period
        end_date = start_date + pd.DateOffset(months=3)
        # Select the segment of data between start_date and end_date
        segment = data[(data['date'] >= start_date) & (data['date'] < end_date)]
        if not segment.empty:
            trend = categorize_trend(segment)
            if trend == "upward":
                upward_segments.append(segment)
            elif trend == "downward":
                downward_segments.append(segment)
            else: # bumpy trend
                bumpy_segments.append(segment)
        # Update start_date to the next segment's start
        start_date = end_date
        # print("trend_cat",trend_categories)
    return trend_categories

In [30]:
stock_data = load_data(dataframes_array_train[0])
trends = analyze_trends(stock_data)

In [24]:
stock_data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.747391,7.460152e+08,4.0,0.000000,2.969345,2.641387,100.000000,66.666667,100.000000,2.747391,2.747391,39.189999,0.000000
1,2009-01-05,AAPL,3.327500,3.435000,3.311071,2.863341,1.181608e+09,0.0,0.002601,2.969345,2.641387,100.000000,66.666667,100.000000,2.805366,2.805366,39.080002,0.000000
2,2009-01-06,AAPL,3.426786,3.470357,3.299643,2.816113,1.289310e+09,1.0,0.001850,2.925561,2.692336,70.355275,46.865686,100.000000,2.808948,2.808948,38.560001,0.000000
3,2009-01-07,AAPL,3.278929,3.303571,3.223571,2.755262,7.530488e+08,2.0,-0.000734,2.904833,2.686220,50.429178,-29.676912,43.607834,2.795527,2.795527,43.389999,0.000000
4,2009-01-08,AAPL,3.229643,3.326786,3.215714,2.806425,6.735008e+08,3.0,-0.000087,2.892869,2.702544,60.226840,-9.099173,48.357918,2.797706,2.797706,42.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,2020-06-24,AAPL,91.250000,92.197502,89.629997,88.141579,1.926232e+08,2.0,3.283724,90.666287,75.768202,63.765180,135.613473,38.191255,81.123170,74.731050,33.840000,10.120450
2889,2020-06-25,AAPL,90.175003,91.250000,89.392502,89.311707,1.375224e+08,3.0,3.330739,91.270883,76.304115,64.828595,123.656623,36.230796,81.589835,75.184889,32.220001,22.286126
2890,2020-06-26,AAPL,91.102501,91.330002,88.254997,86.567528,2.052568e+08,4.0,3.110708,91.249612,77.199067,60.519612,94.940099,27.005877,81.949604,75.647434,34.730000,68.833763
2891,2020-06-29,AAPL,88.312500,90.542503,87.820000,88.562622,1.306460e+08,0.0,3.062023,91.501954,77.924199,62.399269,92.929821,23.584901,82.390809,76.126874,31.780001,36.823301


In [33]:
upward_segments[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
125,2009-07-02,AAPL,5.044643,5.101071,4.992500,4.239004,370479200.0,3.0,0.070504,4.414837,4.028478,58.897686,56.533791,17.658939,4.171654,3.968478,27.950001,0.0
126,2009-07-06,AAPL,4.953571,4.963929,4.866071,4.196318,498688400.0,0.0,0.062215,4.390909,4.034059,57.440633,-3.807250,3.089725,4.186216,3.979725,29.000000,0.0
127,2009-07-07,AAPL,4.945714,4.988571,4.827857,4.099137,461596800.0,1.0,0.047260,4.371676,4.027711,54.278234,-40.855490,0.876093,4.199234,3.987713,30.850000,0.0
128,2009-07-08,AAPL,4.854286,4.930000,4.800714,4.154235,575929200.0,2.0,0.039399,4.354574,4.028161,55.708436,-58.796290,3.689218,4.205733,3.996290,31.299999,0.0
129,2009-07-09,AAPL,4.920000,4.928214,4.854643,4.128201,343025200.0,3.0,0.030715,4.348889,4.022070,54.869496,-57.921912,3.689218,4.209073,4.005398,29.780001,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,2009-09-25,AAPL,6.500357,6.625000,6.480000,5.521118,445239200.0,4.0,0.161972,5.766507,4.874599,66.465383,111.723146,25.195254,5.228105,4.950197,25.610001,0.0
185,2009-09-28,AAPL,6.566786,6.667143,6.547500,5.635557,337444800.0,0.0,0.163541,5.804277,4.885569,68.681429,120.291475,28.176107,5.247652,4.973473,24.879999,0.0
186,2009-09-29,AAPL,6.668929,6.692857,6.582500,5.612246,345385600.0,1.0,0.161047,5.828937,4.912891,67.738169,111.981197,29.995577,5.273678,4.997072,25.190001,0.0
187,2009-09-30,AAPL,6.647500,6.658929,6.521786,5.611338,539585200.0,2.0,0.157185,5.836909,4.965619,67.700708,94.105443,22.183424,5.295223,5.022275,25.610001,0.0


<font size="5">Visualize</font>



In [36]:
def plot_close_price(file, identifier):
    # Load the data
    data = file

    # Convert the 'date' column to datetime
    data['date'] = pd.to_datetime(data['date'])

    start_date = data['date'].iloc[0].strftime("%Y%m%d")

    # Plotting
    plt.close('all')
    plt.figure(figsize=(12, 6))
    plt.plot(data['date'], data['close'], label='Close Price')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title('Close Price over Time')
    plt.legend()
    plt.grid(True)
    plt.show()
    output_file = f'close_price_plot_{start_date}.png'
    plt.savefig(output_file)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
for i in range(len(downward_segments)):
  plot_close_price(downward_segments[i], 'downward')
for i in range(len(bumpy_segments)):
  plot_close_price(bumpy_segments[i], 'bumpy')
for i in range(len(upward_segments)):
  plot_close_price(upward_segments[i], 'upward')


# Construct Stock Trading Environment

In [40]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# Global variables
HMAX_NORMALIZE = 200
INITIAL_ACCOUNT_BALANCE=100000
STOCK_DIM = 1

# transaction fee: 1/1000 reasonable percentage
TRANSACTION_FEE_PERCENT = 0.001
# REWARD_SCALING = 1e-3


class SingleStockEnv(gym.Env):
    """A stock trading environment for OpenAI gym"""
    metadata = {'render.modes': ['human']}

    def __init__(self, df,day = 0):
        #super(StockEnv, self).__init__()
        # date increment
        self.day = day
        self.df = df
        # action_space normalization and the shape is STOCK_DIM
        self.action_space = spaces.Box(low = -1, high = 1,shape = (STOCK_DIM,))
        # Shape = 4: [Current Balance]+[prices]+[owned shares] +[macd]
        self.observation_space = spaces.Box(low=0, high=np.inf, shape = (4,))
        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        # termination
        self.terminal = False
        # save the total number of trades
        self.trades = 0
        # initalize state
        self.state = [INITIAL_ACCOUNT_BALANCE] + \
                      [self.data.rsi_30] + \
                      [0]*STOCK_DIM + \
                      [self.data.macd]
        # initialize reward and cost
        self.reward = 0
        self.cost = 0

        # memorize the total value, total rewards
        self.asset_memory = [INITIAL_ACCOUNT_BALANCE]
        self.rewards_memory = []

    def _sell_stock(self, index, action):
        # perform sell action based on the sign of the action
        if self.state[index+STOCK_DIM+1] > 0:
            # update balance
            self.state[0] += \
            self.state[index+1]*min(abs(action),self.state[index+STOCK_DIM+1]) * \
             (1- TRANSACTION_FEE_PERCENT)
            # update held shares
            self.state[index+STOCK_DIM+1] -= min(abs(action), self.state[index+STOCK_DIM+1])
            # update transaction costs
            self.cost +=self.state[index+1]*min(abs(action),self.state[index+STOCK_DIM+1]) * \
             TRANSACTION_FEE_PERCENT
            self.trades+=1
        else:
            pass

    def _buy_stock(self, index, action):
        # perform buy action based on the sign of the action
        available_amount = self.state[0] // self.state[index+1]
        #update balance
        self.state[0] -= self.state[index+1]*min(available_amount, action)* \
                          (1+ TRANSACTION_FEE_PERCENT)
        # update held shares
        self.state[index+STOCK_DIM+1] += min(available_amount, action)
        # update transaction costs
        self.cost+=self.state[index+1]*min(available_amount, action)* \
                          TRANSACTION_FEE_PERCENT
        self.trades+=1

    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique())-1

        if self.terminal:
            plt.plot(self.asset_memory,'r')
            plt.savefig('account_value.png')
            plt.close()

            end_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))
            print("previous_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(end_total_asset))

            df_total_value = pd.DataFrame(self.asset_memory)
            df_total_value.to_csv('account_value.csv')
            print("total_reward:{}".format(self.state[0]+sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))- INITIAL_ACCOUNT_BALANCE ))
            print("total_cost: ", self.cost)
            print("total trades: ", self.trades)

            df_total_value.columns = ['account_value']
            df_total_value['daily_return']=df_total_value.pct_change(1)

            if df_total_value['daily_return'].std()!=0:
                sharpe = (252**0.5)*df_total_value['daily_return'].mean()/ \
                      df_total_value['daily_return'].std()
                print("Sharpe: ",sharpe)
            df_rewards = pd.DataFrame(self.rewards_memory)
            df_rewards.to_csv('account_rewards.csv')
            return self.state, self.reward, self.terminal,{}

        else:

            # actions are the shares we need to buy, hold, or sell
            actions = actions * HMAX_NORMALIZE
            # calculate begining total asset
            begin_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))

            # perform buy or sell action
            argsort_actions = np.argsort(actions)
            sell_index = argsort_actions[:np.where(actions < 0)[0].shape[0]]
            buy_index = argsort_actions[::-1][:np.where(actions > 0)[0].shape[0]]

            for index in sell_index:
                # print('take sell action'.format(actions[index]))
                self._sell_stock(index, actions[index])

            for index in buy_index:
                # print('take buy action: {}'.format(actions[index]))
                self._buy_stock(index, actions[index])

            # update data, walk a step s'
            self.day += 1
            self.data = self.df.loc[self.day,:]
            #load next state
            self.state =  [self.state[0]] + \
                          [self.data.rsi_30] + \
                          list(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]) +\
                          [self.data.macd]

            # calculate the end total asset
            end_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))
            self.reward = end_total_asset - begin_total_asset
            self.rewards_memory.append(self.reward)
            #self.reward = self.reward * REWARD_SCALING
            self.asset_memory.append(end_total_asset)

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [INITIAL_ACCOUNT_BALANCE]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        self.cost = 0
        self.trades = 0
        self.terminal = False
        self.rewards_memory = []
        #initiate state
        self.state = [INITIAL_ACCOUNT_BALANCE] + \
                      [self.data.rsi_30] + \
                      [0]*STOCK_DIM + \
                      [self.data.macd]
        return self.state

    def render(self, mode='human'):
        return self.state

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

In [53]:
import numpy as np
import gym
from gym import spaces
import pandas as pd

class StockTradingEnv(gym.Env):
    """
    A stock trading environment for OpenAI gym
    """
    def __init__(self, stock_data, initial_balance=10000, lookback_window_size=10):
        super(StockTradingEnv, self).__init__()

        self.stock_data = stock_data
        self.initial_balance = initial_balance
        self.lookback_window_size = lookback_window_size

        self.action_space = spaces.Discrete(3)  # Sell, Hold, Buy
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(lookback_window_size + 5,))  # +X for additional indicators

        self.reset()

    def reset(self):
        self.balance = self.initial_balance
        self.current_step = 0
        self.holdings = 0  # Number of shares held
        return self._next_observation()

    def _next_observation(self):
        # Use data for the last N days
        frame = self.stock_data.iloc[self.current_step:self.current_step + self.lookback_window_size]

        # Append additional features like balance, holdings and technical indicators
        additional_features = self._get_technical_indicators(frame)
        obs = np.append(frame.values.flatten(), [self.balance, self.holdings, *additional_features])
        return obs

    def _get_technical_indicators(self, frame):
        # Calculate and return technical indicators (like moving average, RSI, etc.)
        # Dummy implementation
        return [0.0] * 5  # Replace X with the number of indicators

    def step(self, action):
        current_price = self.stock_data.iloc[self.current_step]['Close']
        self.current_step += 1

        if action == 0:  # Sell
            self.balance += self.holdings * current_price
            self.holdings = 0
        elif action == 2 and self.balance >= current_price:  # Buy
            self.holdings += self.balance // current_price
            self.balance -= self.holdings * current_price

        done = self.current_step >= len(self.stock_data)
        reward = self._calculate_reward()
        obs = self._next_observation()
        return obs, reward, done, {}

    def _calculate_reward(self):
        # Simple reward: Change in portfolio value
        current_portfolio_value = self.balance + self.holdings * self.stock_data.iloc[self.current_step]['Close']
        return current_portfolio_value - self.initial_balance

    def render(self, mode='human', close=False):
        # Optionally implement rendering for visualization
        pass


In [54]:
import random
def step(self, action):
    current_price = self.stock_data[self.current_step]['Close']
    self.current_step += 1

    if action == 0:  # Sell
        # Implement selling logic: update balance based on current_price
        pass
    elif action == 2:  # Buy
        # Implement buying logic: update balance based on current_price
        pass
    # Hold does nothing

    reward = self._calculate_reward()
    done = self.current_step >= len(self.stock_data)
    return self._next_observation(), reward, done, {}

def sample_tasks(stock_data, num_tasks):
    sampled_stocks = random.sample(list(stock_data.keys()), num_tasks)
    return [StockTradingEnv(stock_data[stock]) for stock in sampled_stocks]

In [55]:
import torch
import torch.nn as nn

class PolicyNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PolicyNetwork, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Softmax(dim=-1)
        )

    def forward(self, x):
        return self.layers(x)

policy_net = PolicyNetwork(input_size=30, hidden_size=128, output_size=3)

# Main Training Loop

In [72]:
# Optional: Evaluate the updated policy on validation tasks
K = 20
# ... rest of the functions used above
def initialize_policy_parameters(policy_net):
    # Assuming a policy network class has been defined
    # policy_net = PolicyNetwork(input_size, hidden_size, output_size)
    # return policy_net.state_dict()
    return policy_net.parameters()
def sample_trajectories(task_env, policy_params, num_trajectories=K):
    trajectories = []
    policy_net.load_state_dict(policy_params)  # Assuming policy_net is a global variable for the policy network

    for _ in range(num_trajectories):
        state = task_env.reset()
        trajectory = []
        while not done:
            action = policy_net.act(state)  # Assuming a method that decides an action based on the current state
            next_state, reward, done, _ = task_env.step(action)
            trajectory.append((state, action, reward))
            state = next_state
        trajectories.append(trajectory)
    return trajectories
def compute_loss(trajectories, policy_params):
    policy_losses = []
    policy_net.load_state_dict(policy_params)  # Load the current policy parameters

    for trajectory in trajectories:
        for state, action, reward in trajectory:
            # Compute the policy loss from the trajectory data
            # This could use a function like policy_net.evaluate_actions
            log_prob = policy_net.evaluate_actions(state, action)
            policy_losses.append(-log_prob * reward)  # Negative because we perform gradient ascent
    return torch.stack(policy_losses).sum()
def compute_meta_gradient(tasks, D_prime, theta, theta_i_prime, task_gradients):
    meta_gradient = torch.zeros_like(theta)

    for i, task in enumerate(tasks):
        # Load the adapted parameters for the current task
        policy_net.load_state_dict(theta_i_prime[i])

        # Sample new trajectories using the adapted parameters
        new_trajectories = sample_trajectories(task, theta_i_prime[i], num_trajectories=K)

        # Compute the loss with the new trajectories
        new_loss = compute_loss(new_trajectories, theta_i_prime[i])

        # Compute the gradient w.r.t. the adapted parameters
        adapted_gradients = torch.autograd.grad(new_loss, theta_i_prime[i])

        # Combine the adapted gradient with the initial gradient for the meta-gradient
        meta_gradient += adapted_gradients[0] - task_gradients[i]

    return meta_gradient / len(tasks)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [73]:
from copy import deepcopy
alpha = 1e-3
beta = 1e-4
theta = initialize_policy_parameters(policy_net)
meta_optimizer = torch.optim.Adam(theta, lr=beta)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [74]:
print(theta)

<generator object Module.parameters at 0x7e3818f68900>


In [66]:
# Initialization
epochs = 10
num_tasks = 1

# Meta-training loop
for i in range(epochs):
    task_gradients = []  # To store gradients for each task

    # Task sampling
    tasks = sample_tasks(stock_data,num_tasks)

    # Inner loop
    for task in tasks:
        # Copy initial policy parameters for task-specific learning
        theta_i = deepcopy(theta)

        # Sample K trajectories using the current policy parameters theta
        D = sample_trajectories(task, theta_i, num_trajectories=K)

        # Evaluate gradients of the loss w.r.t. theta_i
        task_loss = compute_loss(D, theta_i)
        task_gradients.append(torch.autograd.grad(task_loss, theta_i))

        # Compute adapted parameters theta'_i using gradient descent
        theta_i_prime = theta_i - alpha * task_gradients[-1]

        # Sample new trajectories D'_i using the adapted parameters theta'_i
        D_prime = sample_trajectories(task, theta_i_prime, num_trajectories=K)

    # Outer loop
    meta_gradient = compute_meta_gradient(tasks, D_prime, theta, theta_i_prime, task_gradients)
    meta_optimizer.zero_grad()
    theta.grad = meta_gradient
    meta_optimizer.step()


TypeError: ignored